In [2]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from utils import loader
from utils import debugger 
from utils import preprocessing as pp 
from utils import misc
from utils import constant
import implementations as impl
import cost
import model

In [4]:
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, x, ids = loader.load_csv_data(DATA_TRAIN_PATH)

In [5]:
to_replace = [(constant.UNDEF_VAL, 'most_frequent')]


x_split, y_split = pp.preprocess_jet_num(x, y, to_replace, do_normalise=False, add_bias=True)

In [6]:
data_part = []

nb_samples = []
for x_, y_ in zip(x_split, y_split):
    train_data, test_data = misc.random_data_split(y_, x_, k_fold=10)
    data = {'train': train_data, 'test': test_data}
    data_part.append(data)
    nb_samples.append(len(test_data[1]))

In [7]:
models = []

learning_param = {
    'max_iters' : 200,
    'gamma': 1e-6,
    'lambda_': 1e-6
}


model_list = ['ridge_regression', 'ridge_regression', 'ridge_regression', 'ridge_regression']
acc_list = []

for idx in range(len(data_part)):
    tx, y = data_part[idx]['train']
    nb_features = tx.shape[1]
    
    w_initial = np.random.uniform(0,1,size=nb_features) 
    model_ls = model.Model(model_list[idx], w_initial, learning_param, debug=True)
    
    model_ls.learn_weights(y, tx)
    models.append(model_ls)
    
    tx_te, y_te = data_part[idx]['test']
    y_ = model_ls.predict(tx_te)
    acc_list.append(misc.accuracy(y_te, y_))

learning weights...
done.
learning weights...
done.
learning weights...
done.
learning weights...
done.


In [8]:
avg_acc = np.dot(acc_list, nb_samples) / sum(nb_samples)
avg_acc

0.7588607088567085

In [9]:
acc_list, sum(nb_samples), nb_samples

([0.8190371334200781,
  0.7126644312612845,
  0.7297994838197339,
  0.7152527075812274],
 24998,
 [9991, 7754, 5037, 2216])

In [10]:


w_initial = np.random.uniform(0,1,size=nb_features) 

learning_param = {
    'lambda_': 10e-5
}


model_ls = model.Model('ridge_regression', w_initial, learning_param, debug=True)

In [11]:
model_ls.learn_weights(y, tx)

learning weights...
done.


In [12]:
y_ = model_ls.predict(tx)
print('Train accuracy', misc.accuracy(y, y_))

Train accuracy 0.7231247492980345


### Test :
1. Load the test data
2. preprocess with jet num
3. predict the 4 splitted x with their respective model

In [13]:
DATA_TEST_PATH = '../data/test.csv'
_, x_new, ids = loader.load_csv_data(DATA_TEST_PATH)

In [14]:
# to_replace = [(constant.UNDEF_VAL, 'most_frequent')]

x_new_split, _ = pp.preprocess_jet_num(x_new, None, to_replace, do_normalise=False, add_bias=True)


In [15]:
y_new_split = []
for i,x_sub in enumerate(x_new_split):
    y_new_split.append(models[i].predict(x_sub))

In [16]:
y_new_split

[array([-1., -1.,  1., ..., -1.,  1., -1.]),
 array([-1., -1., -1., ..., -1., -1., -1.]),
 array([-1.,  1., -1., ..., -1., -1.,  1.]),
 array([-1., -1., -1., ..., -1., -1., -1.])]